In [1]:
!pip install -q ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 23.8 MB/s eta 0:00:00


# Import Library

In [2]:
import numpy as np
import pandas as pd 
from ultralytics import YOLO
import cv2
import json
import os
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import warnings
warnings.filterwarnings('ignore')        

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
/kaggle/input/best/pytorch/default/1/best.pt
/kaggle/input/dataset-video/15sec_input_720p.mp4
/kaggle/input/dataset-video/tacticam.mp4
/kaggle/input/dataset-video/broadcast.mp4


# Paths

In [3]:
# Paths
broadcast_path = '/kaggle/input/dataset-video/broadcast.mp4'
tacticam_path = '/kaggle/input/dataset-video/tacticam.mp4'
model_path = '/kaggle/input/dataset-video/best.pt'

# Output directories
os.makedirs("/kaggle/working/outputs/detections", exist_ok=True)
os.makedirs("/kaggle/working/outputs/features", exist_ok=True)
os.makedirs("/kaggle/working/outputs/matches", exist_ok=True)


# Load YOLOv11

In [4]:
# Load model
model = YOLO('/kaggle/input/best/pytorch/default/1/best.pt')

# Detection Function

In [5]:
def detect_players(video_path, output_json, frame_skip=5):
    cap = cv2.VideoCapture(video_path)
    frame_id = 0
    detections = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_id % frame_skip == 0:
            results = model.predict(frame, conf=0.3, verbose=False)[0]
            for box in results.boxes:
                cls = int(box.cls[0])
                if cls == 0:  # class 0 = player
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    conf = float(box.conf[0])
                    detections.append({
                        "frame_id": frame_id,
                        "bbox": [x1, y1, x2, y2],
                        "confidence": conf
                    })

        frame_id += 1

    with open(output_json, 'w') as f:
        json.dump(detections, f)

    cap.release()

# Feature Extraction

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

resnet = models.resnet50(pretrained=True)
resnet = torch.nn.Sequential(*list(resnet.children())[:-1])  # remove final layer
resnet = resnet.to(device).eval()

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

def extract_features(video_path, detection_json):
    cap = cv2.VideoCapture(video_path)
    with open(detection_json) as f:
        detections = json.load(f)

    features = []
    for det in detections:
        cap.set(cv2.CAP_PROP_POS_FRAMES, det['frame_id'])
        ret, frame = cap.read()
        if not ret:
            continue
        x1, y1, x2, y2 = det['bbox']
        crop = frame[y1:y2, x1:x2]
        if crop.size == 0:
            continue
        img = Image.fromarray(cv2.cvtColor(crop, cv2.COLOR_BGR2RGB))
        img_tensor = transform(img).unsqueeze(0).to(device)

        with torch.no_grad():
            feat = resnet(img_tensor).squeeze().cpu().numpy()
        features.append(feat)

    cap.release()
    return np.array(features)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 228MB/s]


# Match Players

In [7]:
def match_players(broadcast_feats, tacticam_feats):
    matched_ids = []
    for i, tf in enumerate(tacticam_feats):
        sims = cosine_similarity([tf], broadcast_feats)[0]
        best_match = int(np.argmax(sims))
        matched_ids.append({
            "tacticam_id": i,
            "matched_broadcast_id": best_match,
            "similarity": float(sims[best_match])
        })
    return matched_ids

# Run Full Pipeline

In [8]:
# Detection
detect_players(broadcast_path, "/kaggle/working/outputs/detections/broadcast.json", frame_skip=5)
detect_players(tacticam_path, "/kaggle/working/outputs/detections/tacticam.json", frame_skip=5)

# Feature Extraction
broadcast_feats = extract_features(broadcast_path, "/kaggle/working/outputs/detections/broadcast.json")
tacticam_feats = extract_features(tacticam_path, "/kaggle/working/outputs/detections/tacticam.json")

# Matching
results = match_players(broadcast_feats, tacticam_feats)

# Save Results
with open("/kaggle/working/outputs/matches/match1.json", "w") as f:
    json.dump(results, f, indent=2)

# Print Sample
for r in results[:5]:
    print(f"Tacticam ID {r['tacticam_id']} → Broadcast ID {r['matched_broadcast_id']} (Sim: {r['similarity']:.2f})")

Tacticam ID 0 → Broadcast ID 3 (Sim: 0.89)
Tacticam ID 1 → Broadcast ID 1 (Sim: 0.88)
Tacticam ID 2 → Broadcast ID 0 (Sim: 0.90)
